In [304]:
import pandas as pd
import numpy as np
import requests
import random
import math
import matplotlib.pyplot as plt
from scipy import stats
from scipy import stats, special
from sklearn import model_selection, metrics, linear_model, datasets, feature_selection

In [409]:
PGA_base_url = "https://statdata.pgatour.com/r/" 

#Check PGA for current tournament

PGA_current= "https://statdata.pgatour.com/r/current/message.json"
r = requests.get(PGA_current)
r_json = r.json()
current_id = r_json['tid']
current_url = "https://statdata.pgatour.com/r/{0}".format(current_id)

#Current tournament's JSON

PGA_current_url = "{0}/leaderboard-v2mini.json".format(current_url)

r = requests.get(PGA_current_url)
r_json = r.json()
current_tournament = r_json["debug"]["tournament_in_schedule_file_name"]
print(current_tournament)
print(PGA_current_url)

U.S. Open
https://statdata.pgatour.com/r/026/leaderboard-v2mini.json


In [329]:
team1 = "Rob"
team1_list = {"Tim Wilkinson", "Bill Haas", "Jhonattan Vegas"}

team2 = "John"
team2_list = {"Matthieu Pavon", "Cameron Wilson", "Brandt Snedeker"}

team3 = "Joe"
team3_list = {"Kiradech Aphibarnrat", "Steve Stricker", "Gary Woodland"}


teams = {team1 : team1_list, team2 : team2_list, team3 : team3_list}

In [427]:
PGA_base_url = "https://statdata.pgatour.com/r/" 

#Check PGA for current tournament

PGA_current= "https://statdata.pgatour.com/r/current/message.json"
r = requests.get(PGA_current)
r_json = r.json()
current_id = r_json['tid']
current_url = "https://statdata.pgatour.com/r/{0}".format(current_id)

#Current tournament's JSON

PGA_current_url = "{0}/leaderboard-v2mini.json".format(current_url)

r = requests.get(PGA_current_url)
r_json = r.json()
current_tournament = r_json["debug"]["tournament_in_schedule_file_name"]

#Create Scoreboard

field = 156
score = r_json["leaderboard"]["players"]

team_list = []

for team in teams.keys():
    players = teams[team]
    for player in players:
        name = player.split()
        for i in range(0, field):
            try:
                if name[0] == score[i]["player_bio"]["first_name"]:
                    if name[1] == score[i]["player_bio"]["last_name"]:
                        row_dict = {"Team": team,
                                "Player": player,
                                'Today': score[i]["today"],
                                'Total' : score[i]["total"],
                                'Thru': score[i]["thru"]}
                        team_list.append(row_dict)

            except:
                print(i)

df = pd.DataFrame(team_list)
df = df.set_index("Team")
df = df[["Player","Today","Total","Thru"]]


#create team score

score_list = []
for team in teams.keys():
    score = 0
    count = -1 
    for i in list(df.index):
        count = count + 1
        if i == team:
            score = score + df.iloc[count][1]
    score_dict = {"Team": team, "Score": score}
    score_list.append(score_dict)        
    
df_score = pd.DataFrame(score_list)
df_score = df_score.set_index("Team")

df["Score"] = df_score

#clean scores

for i in range(0,len(df.index)):
    if i % len(list(teams[team1])) != 0:
        df.iloc[i, df.columns.get_loc('Score')] = ""
        
#drop the worst score

    else:
        df.iloc[i, df.columns.get_loc('Score')] = df.iloc[i][4] - df.iloc[i:i+len(list(teams[team1]))]["Today"].max()


df

,Player,Today,Total,Thru,Score
Team,,,,,
Rob,Tim Wilkinson,8,16,14,3
Rob,Bill Haas,2,10,12,
Rob,Jhonattan Vegas,1,9,12,
John,Cameron Wilson,1,9,11,2
John,Matthieu Pavon,1,9,11,
John,Brandt Snedeker,4,12,10,
Joe,Gary Woodland,0,8,9,-2
Joe,Steve Stricker,0,8,9,
Joe,Kiradech Aphibarnrat,-2,6,10,
